# 🚀 GraphRAG GPU检测与测试 - Google Colab版本

本Notebook用于在Google Colab上检测GPU可用性并测试GraphRAG系统的性能。

## 📋 使用步骤

1. **启用GPU**: 运行时 → 更改运行时类型 → 硬件加速器 → GPU (T4)
2. **运行所有单元格**: 依次执行下面的代码
3. **查看结果**: 检查GPU加速效果

---

## 1️⃣ GPU环境检测

In [ ]:
# 检测GPU可用性
import torch
import subprocess
import sys

print("="*60)
print("🔍 GPU环境检测")
print("="*60)

# PyTorch GPU检测
cuda_available = torch.cuda.is_available()
print(f"\n✅ CUDA可用: {cuda_available}")

if cuda_available:
    print(f"   GPU数量: {torch.cuda.device_count()}")
    print(f"   当前GPU: {torch.cuda.current_device()}")
    print(f"   GPU名称: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA版本: {torch.version.cuda}")
    
    # 显存信息
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"   总显存: {total_memory:.2f} GB")
    
    # nvidia-smi信息
    print("\n📊 nvidia-smi 输出:")
    print("-"*60)
    !nvidia-smi
else:
    print("\n⚠️  警告: 未检测到GPU")
    print("   请检查: 运行时 → 更改运行时类型 → 硬件加速器 → GPU")

print("\n" + "="*60)

## 2️⃣ GPU性能基准测试

In [ ]:
# GPU vs CPU 性能对比
import time
import numpy as np

print("="*60)
print("⚡ GPU vs CPU 矩阵运算性能测试")
print("="*60)

# 测试参数
matrix_size = 5000

# CPU测试
print(f"\n🔵 CPU测试 (矩阵大小: {matrix_size}x{matrix_size})")
a_cpu = torch.randn(matrix_size, matrix_size)
b_cpu = torch.randn(matrix_size, matrix_size)

start = time.time()
c_cpu = torch.mm(a_cpu, b_cpu)
cpu_time = time.time() - start
print(f"   CPU时间: {cpu_time:.2f} 秒")

# GPU测试
if cuda_available:
    print(f"\n🟢 GPU测试 (矩阵大小: {matrix_size}x{matrix_size})")
    a_gpu = torch.randn(matrix_size, matrix_size).cuda()
    b_gpu = torch.randn(matrix_size, matrix_size).cuda()
    
    # 预热GPU
    _ = torch.mm(a_gpu, b_gpu)
    torch.cuda.synchronize()
    
    start = time.time()
    c_gpu = torch.mm(a_gpu, b_gpu)
    torch.cuda.synchronize()
    gpu_time = time.time() - start
    print(f"   GPU时间: {gpu_time:.2f} 秒")
    
    speedup = cpu_time / gpu_time
    print(f"\n🚀 加速比: {speedup:.2f}x")
    print(f"   GPU比CPU快 {speedup:.1f} 倍!")
else:
    print("\n⚠️  跳过GPU测试（GPU不可用）")

print("\n" + "="*60)

## 3️⃣ 安装GraphRAG依赖

In [ ]:
# 克隆项目（如果需要）
import os

print("📦 安装GraphRAG依赖...\n")

# 安装核心依赖
!pip install -q langchain langchain-community langchain-core langgraph
!pip install -q chromadb sentence-transformers transformers
!pip install -q tiktoken beautifulsoup4 requests
!pip install -q tavily-python python-dotenv
!pip install -q networkx python-louvain
!pip install -q torch --index-url https://download.pytorch.org/whl/cu118

print("\n✅ 依赖安装完成!")

## 4️⃣ 上传项目文件

**选项A**: 从GitHub克隆
```python
!git clone https://github.com/your-repo/adaptive_RAG.git
%cd adaptive_RAG
```

**选项B**: 手动上传文件到Colab
- 使用左侧文件浏览器上传以下核心文件：
  - `config.py`
  - `entity_extractor.py`
  - `knowledge_graph.py`
  - `graph_indexer.py`
  - `graph_retriever.py`
  - `.env` (包含API密钥)

In [ ]:
# 创建必要的目录
!mkdir -p data

# 如果使用选项A，运行下面的命令
# !git clone YOUR_REPO_URL
# %cd adaptive_RAG

print("✅ 目录准备完成")

## 5️⃣ 配置API密钥

In [ ]:
# 设置API密钥（替换为您的真实密钥）
import os
from getpass import getpass

print("🔑 配置API密钥\n")

# 方式1: 直接设置（不安全，仅用于测试）
# os.environ['TAVILY_API_KEY'] = 'your_tavily_api_key_here'

# 方式2: 安全输入
if 'TAVILY_API_KEY' not in os.environ:
    os.environ['TAVILY_API_KEY'] = getpass('输入 TAVILY_API_KEY: ')
    print("✅ TAVILY_API_KEY 已设置")
else:
    print("✅ TAVILY_API_KEY 已存在")

print("\n注意: GraphRAG在Colab上使用HuggingFace嵌入，不需要NOMIC_API_KEY")

## 6️⃣ 简化版GraphRAG测试代码

In [ ]:
# 简化版GraphRAG核心组件
# 适用于Colab快速测试，无需完整项目文件

from typing import List, Dict
import networkx as nx
from sentence_transformers import SentenceTransformer
import torch

class SimpleGraphRAG:
    """简化版GraphRAG用于GPU性能测试"""
    
    def __init__(self, use_gpu=True):
        print("🚀 初始化SimpleGraphRAG...")
        
        # 检测设备
        self.device = 'cuda' if use_gpu and torch.cuda.is_available() else 'cpu'
        print(f"   设备: {self.device.upper()}")
        
        # 加载嵌入模型
        print(f"   加载嵌入模型...")
        self.embedder = SentenceTransformer(
            'sentence-transformers/all-MiniLM-L6-v2',
            device=self.device
        )
        
        # 知识图谱
        self.graph = nx.Graph()
        self.entities = {}
        
        print("✅ 初始化完成!")
    
    def add_sample_data(self):
        """添加示例数据"""
        print("\n📊 添加示例数据...")
        
        # 示例实体
        entities = [
            {"name": "LLM", "type": "CONCEPT", "desc": "大语言模型"},
            {"name": "GPT", "type": "TECHNOLOGY", "desc": "生成式预训练转换器"},
            {"name": "Transformer", "type": "CONCEPT", "desc": "注意力机制架构"},
            {"name": "OpenAI", "type": "ORGANIZATION", "desc": "人工智能研究公司"},
            {"name": "Attention", "type": "CONCEPT", "desc": "注意力机制"},
        ]
        
        for entity in entities:
            self.graph.add_node(
                entity["name"],
                type=entity["type"],
                description=entity["desc"]
            )
            self.entities[entity["name"]] = entity
        
        # 示例关系
        relations = [
            ("GPT", "LLM", "IS_A"),
            ("GPT", "Transformer", "USES"),
            ("Transformer", "Attention", "CONTAINS"),
            ("OpenAI", "GPT", "DEVELOPS"),
        ]
        
        for source, target, rel_type in relations:
            self.graph.add_edge(source, target, relation=rel_type)
        
        print(f"   ✅ 添加了 {len(entities)} 个实体")
        print(f"   ✅ 添加了 {len(relations)} 个关系")
    
    def test_gpu_embedding(self, texts: List[str]):
        """测试GPU嵌入性能"""
        print(f"\n⚡ 测试嵌入性能 ({len(texts)} 个文本)...")
        
        import time
        
        start = time.time()
        embeddings = self.embedder.encode(
            texts,
            show_progress_bar=True,
            batch_size=32
        )
        elapsed = time.time() - start
        
        print(f"   ✅ 完成! 耗时: {elapsed:.2f}秒")
        print(f"   📊 嵌入维度: {embeddings.shape}")
        print(f"   🚀 速度: {len(texts)/elapsed:.1f} 文本/秒")
        
        return embeddings
    
    def query(self, question: str):
        """简单查询"""
        print(f"\n🔍 查询: {question}")
        
        # 简单的关键词匹配
        results = []
        for entity_name in self.entities:
            if entity_name.lower() in question.lower():
                neighbors = list(self.graph.neighbors(entity_name))
                results.append({
                    "entity": entity_name,
                    "info": self.entities[entity_name],
                    "neighbors": neighbors
                })
        
        print(f"\n📋 找到 {len(results)} 个相关实体:")
        for r in results:
            print(f"   • {r['entity']} ({r['info']['type']})")
            print(f"     描述: {r['info']['desc']}")
            print(f"     关联: {', '.join(r['neighbors'])}")
        
        return results

print("✅ SimpleGraphRAG类定义完成")

## 7️⃣ 运行GPU性能测试

In [ ]:
# 初始化GraphRAG（GPU版本）
print("="*60)
print("🎯 GraphRAG GPU性能测试")
print("="*60)

graph_rag = SimpleGraphRAG(use_gpu=True)

# 添加示例数据
graph_rag.add_sample_data()

# 准备测试文本
test_texts = [
    "Large Language Models are transforming AI",
    "GPT uses Transformer architecture",
    "Attention mechanism is key to modern NLP",
    "OpenAI develops cutting-edge AI models",
] * 25  # 100个文本

print(f"\n准备了 {len(test_texts)} 个测试文本")

# GPU嵌入测试
embeddings = graph_rag.test_gpu_embedding(test_texts)

# 测试查询
graph_rag.query("What is GPT?")
graph_rag.query("Tell me about Transformer")

print("\n" + "="*60)
print("✅ GPU性能测试完成!")
print("="*60)

## 8️⃣ CPU vs GPU 性能对比

In [ ]:
# CPU vs GPU 嵌入性能对比
import time

print("="*60)
print("📊 CPU vs GPU 嵌入性能对比")
print("="*60)

# 准备大量测试文本
large_test_texts = test_texts * 10  # 1000个文本
print(f"\n测试数据: {len(large_test_texts)} 个文本\n")

# CPU测试
print("🔵 CPU测试...")
graph_rag_cpu = SimpleGraphRAG(use_gpu=False)
start = time.time()
embeddings_cpu = graph_rag_cpu.embedder.encode(
    large_test_texts,
    show_progress_bar=False,
    batch_size=32
)
cpu_time = time.time() - start
print(f"   CPU时间: {cpu_time:.2f}秒")
print(f"   速度: {len(large_test_texts)/cpu_time:.1f} 文本/秒")

# GPU测试
if cuda_available:
    print("\n🟢 GPU测试...")
    graph_rag_gpu = SimpleGraphRAG(use_gpu=True)
    start = time.time()
    embeddings_gpu = graph_rag_gpu.embedder.encode(
        large_test_texts,
        show_progress_bar=False,
        batch_size=32
    )
    gpu_time = time.time() - start
    print(f"   GPU时间: {gpu_time:.2f}秒")
    print(f"   速度: {len(large_test_texts)/gpu_time:.1f} 文本/秒")
    
    speedup = cpu_time / gpu_time
    print(f"\n🚀 加速比: {speedup:.2f}x")
    print(f"   GPU比CPU快 {speedup:.1f} 倍!")
    
    # 节省的时间
    time_saved = cpu_time - gpu_time
    print(f"   ⏱️  节省时间: {time_saved:.2f}秒")
else:
    print("\n⚠️  GPU不可用，跳过GPU测试")

print("\n" + "="*60)

## 9️⃣ 显存使用监控

In [ ]:
# 监控GPU显存使用
if cuda_available:
    print("="*60)
    print("💾 GPU显存使用情况")
    print("="*60)
    
    allocated = torch.cuda.memory_allocated(0) / (1024**3)
    reserved = torch.cuda.memory_reserved(0) / (1024**3)
    total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    
    print(f"\n已分配: {allocated:.2f} GB")
    print(f"已保留: {reserved:.2f} GB")
    print(f"总显存: {total:.2f} GB")
    print(f"使用率: {(allocated/total)*100:.1f}%")
    
    print("\n详细信息:")
    print(torch.cuda.memory_summary(0, abbreviated=True))
    
    print("\n" + "="*60)
else:
    print("⚠️  GPU不可用")

## 🔟 性能总结报告

In [ ]:
# 生成性能报告
print("="*60)
print("📈 GraphRAG GPU性能测试报告")
print("="*60)

print("\n🖥️  硬件信息:")
if cuda_available:
    print(f"   GPU型号: {torch.cuda.get_device_name(0)}")
    print(f"   显存: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
    print(f"   CUDA版本: {torch.version.cuda}")
else:
    print("   ⚠️  GPU不可用")

print(f"\n   PyTorch版本: {torch.__version__}")
print(f"   Python版本: {sys.version.split()[0]}")

print("\n⚡ 性能测试结果:")
print(f"   矩阵运算加速: ~{speedup if cuda_available else 'N/A'}x")
print(f"   文本嵌入加速: ~{cpu_time/gpu_time if cuda_available else 'N/A'}x")

print("\n💡 建议:")
if cuda_available:
    print("   ✅ GPU运行良好！建议在Colab上运行完整的GraphRAG索引构建")
    print("   ✅ 预计索引构建时间将大幅缩短")
    print("   ✅ 可以处理更大规模的文档集")
else:
    print("   ⚠️  建议启用GPU以获得最佳性能")
    print("   ⚠️  路径: 运行时 → 更改运行时类型 → GPU")

print("\n" + "="*60)
print("✅ 测试完成!")
print("="*60)

---

## 📚 下一步

如果GPU测试成功，您可以：

1. **上传完整项目**: 将整个adaptive_RAG项目上传到Colab
2. **运行GraphRAG索引**: 使用GPU加速构建知识图谱
3. **保存结果**: 将构建好的图谱下载到本地

### 运行完整GraphRAG的命令:

```python
# 上传项目后运行
!python main_graphrag.py
```

### 预期加速效果:

- 实体提取: 使用GPU的LLM推理会更快
- 文本嵌入: **5-10倍加速**
- 向量相似度计算: **10-20倍加速**
- 总体索引构建时间: **3-5倍加速**

---